#Planificación – Caso 1.4

##Dependencias

In [9]:
import sys, subprocess
try:
    import pulp  # noqa: F401
except Exception:
    subprocess.check_call([sys.executable, "-m", "pip", "-q", "install", "pulp"])
    import pulp  # noqa: F401

import pandas as pd
import numpy as np
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, LpStatus, value


Datos del caso 1.4 son: 3 ubicaciones (i=1,2,3); 3 clientes (j=1,2,3); 2 tamaños (small/large), con costes fijos (construcción+ mantenimiento) y costes de transporte, capacidades y demandas.

##DATOS

In [10]:
# Conjuntos
I = [1, 2, 3]          # ubicaciones
J = [1, 2, 3]          # clientes
K = ["S", "L"]         # capacidades (small, large)

# Construcción planta
build_cost = {"S": 100, "L": 120}
cap = {"S": 50, "L": 80}

# Coste mantenimiento por (ubicación, tamaño)
maint = {
    (1, "S"): 20, (1, "L"): 30,
    (2, "S"): 10, (2, "L"): 30,
    (3, "S"): 15, (3, "L"): 40
}

# Coste transporte por (ubicación, tamaño, cliente)
c = {
    (1, "S", 1): 20, (1, "S", 2): 10, (1, "S", 3): 30,
    (1, "L", 1): 18, (1, "L", 2):  8, (1, "L", 3): 27,
    (2, "S", 1): 10, (2, "S", 2): 15, (2, "S", 3): 20,
    (2, "L", 1):  9, (2, "L", 2): 13, (2, "L", 3): 18,
    (3, "S", 1): 15, (3, "S", 2): 15, (3, "S", 3): 20,
    (3, "L", 1): 14, (3, "L", 2): 14, (3, "L", 3): 18,
}

# Demanda
D = {1: 50, 2: 40, 3: 70}

# Coste fijo anual f_{k,i}: "Coste fijo anual" + existe coste construcción.
f = {(i, k): build_cost[k] + maint[(i, k)] for i in I for k in K}


#CREO EL MODELO PuLP

In [11]:
def print_status(prob):
    print("Estado:", LpStatus[prob.status])
    print("Objetivo (coste mínimo):", value(prob.objective))

def flows_to_df(x_vars):
    rows = []
    for (i, k, j), var in x_vars.items():
        rows.append({"Ubicacion": i, "Tam": k, "Cliente": j, "x": value(var)})
    df = pd.DataFrame(rows)
    df["x"] = df["x"].fillna(0.0)
    return df


In [12]:
prob4 = LpProblem("Caso_1_4_Ubicacion_Capacidad", LpMinimize)

# Variables
# y[i,k] = 1 si construimos tamaño k en ubicación i
y = {(i, k): LpVariable(f"y_{i}_{k}", cat="Binary") for i in I for k in K}

# x[i,k,j] = cantidad enviada desde (i,k) a cliente j
x = {(i, k, j): LpVariable(f"x_{i}_{k}_{j}", lowBound=0)
     for i in I for k in K for j in J}

# Función objetivo: sum f_{i,k} y_{i,k} + sum c_{i,k,j} x_{i,k,j}
prob4 += (
    lpSum(f[(i, k)] * y[(i, k)] for i in I for k in K)
    + lpSum(c[(i, k, j)] * x[(i, k, j)] for i in I for k in K for j in J)
)


##Añado las restricciones

In [13]:
# (1) Capacidad: sum_j x_{i,k,j} <= y_{i,k} * C_k
for i in I:
    for k in K:
        prob4 += lpSum(x[(i, k, j)] for j in J) <= y[(i, k)] * cap[k], f"cap_{i}_{k}"

# (2) Demanda: sum_{i,k} x_{i,k,j} = D_j
for j in J:
    prob4 += lpSum(x[(i, k, j)] for i in I for k in K) == D[j], f"dem_{j}"

# (3) Solo 1 capacidad por localización: sum_k y_{i,k} <= 1
for i in I:
    prob4 += lpSum(y[(i, k)] for k in K) <= 1, f"una_cap_{i}"


Resuelvo:

In [15]:
print(prob4)
prob4.solve()
print_status(prob4)


Caso_1_4_Ubicacion_Capacidad:
MINIMIZE
18*x_1_L_1 + 8*x_1_L_2 + 27*x_1_L_3 + 20*x_1_S_1 + 10*x_1_S_2 + 30*x_1_S_3 + 9*x_2_L_1 + 13*x_2_L_2 + 18*x_2_L_3 + 10*x_2_S_1 + 15*x_2_S_2 + 20*x_2_S_3 + 14*x_3_L_1 + 14*x_3_L_2 + 18*x_3_L_3 + 15*x_3_S_1 + 15*x_3_S_2 + 20*x_3_S_3 + 150*y_1_L + 120*y_1_S + 150*y_2_L + 110*y_2_S + 160*y_3_L + 115*y_3_S + 0.0
SUBJECT TO
cap_1_S: x_1_S_1 + x_1_S_2 + x_1_S_3 - 50 y_1_S <= 0

cap_1_L: x_1_L_1 + x_1_L_2 + x_1_L_3 - 80 y_1_L <= 0

cap_2_S: x_2_S_1 + x_2_S_2 + x_2_S_3 - 50 y_2_S <= 0

cap_2_L: x_2_L_1 + x_2_L_2 + x_2_L_3 - 80 y_2_L <= 0

cap_3_S: x_3_S_1 + x_3_S_2 + x_3_S_3 - 50 y_3_S <= 0

cap_3_L: x_3_L_1 + x_3_L_2 + x_3_L_3 - 80 y_3_L <= 0

dem_1: x_1_L_1 + x_1_S_1 + x_2_L_1 + x_2_S_1 + x_3_L_1 + x_3_S_1 = 50

dem_2: x_1_L_2 + x_1_S_2 + x_2_L_2 + x_2_S_2 + x_3_L_2 + x_3_S_2 = 40

dem_3: x_1_L_3 + x_1_S_3 + x_2_L_3 + x_2_S_3 + x_3_L_3 + x_3_S_3 = 70

una_cap_1: y_1_L + y_1_S <= 1

una_cap_2: y_2_L + y_2_S <= 1

una_cap_3: y_3_L + y_3_S <= 1

VARIABLES
x_

##¿Qué plantas se construyen?



In [16]:
print("Plantas construidas (y = 1):")
for i in I:
    for k in K:
        if value(y[(i, k)]) > 0.5:
            print(f" - Ubicación {i}, tamaño {k}, capacidad {cap[k]}, coste fijo {f[(i,k)]}")


Plantas construidas (y = 1):
 - Ubicación 1, tamaño L, capacidad 80, coste fijo 150
 - Ubicación 2, tamaño L, capacidad 80, coste fijo 150
 - Ubicación 3, tamaño L, capacidad 80, coste fijo 160


El modelo ha decidido construir las 3 ubicaciones

En todas, la capacidad L (80 unidades)

Costes fijos:

Ubicación 1 → 150

Ubicación 2 → 150

Ubicación 3 → 160

Esto es coherete con:

Demanda total = 50 + 40 + 70 = 160

Capacidad total instalada = 3 × 80 = 240 que es suficiente.

Y los costes de transporte os permite abrir las 3 que estaría bien ya que el enunnciado solo prohíbe dos tamaños en la misma ubicación.

##Envíos

In [17]:
df_flows = flows_to_df(x)

# Nos quedamos solo con envíos positivos
df_flows = df_flows[df_flows["x"] > 1e-6]

df_flows


,Ubicacion,Tam,Cliente,x
4,1,L,2,40.0
9,2,L,1,50.0
17,3,L,3,70.0


El modelo de programación lineal determina que la solución óptima consiste en construir una planta de tamaño L en las tres ubicaciones disponibles.
Abastecer al cliente 1 desde la ubicación 2, al cliente 2 desde la ubicación 1 y al cliente 3 desde la ubicación 3.

El coste total mínimo obtenido es 2490 euros o dólares.

La solución satisface todas las restricciones de capacidad, demanda y localización del PPL.
